# DSP Test Suite
**Note:** This test suite should be executed in the xsim folder of the Vivado project so that no absolute file paths have to be specified. The path to the xsim folder looks something like this: DSPE2_test\DSPE2_test.sim\sim_1\behav\xsim
### Set up dependencies

In [2]:
from bitstring import BitArray, BitStream  # for bit manipulation and parsing
import numpy as np # for array arithmetic
import tkinter  # for TCL scripting
import pandas as pd # for data frames
import matplotlib.pyplot as plt # for plotting
import subprocess # os stuff
import os

### Generate test vectors for the VHDL test bench
* define test data as bit strings
* put test data in dataframe
* write dataframe to txt
* convert test data to dataframe with signed integers

In [44]:
# define test data vectors. W has a precision of 3 bit!
test_data = {'a1': ['1010', '1111', '0001', '1001', '1001', '0001', '1101'], 
             'a2': ['0111', '1111', '0000', '1001', '1001', '0000', '0001'],
             'w1': [ '001',  '111',  '010',  '000',  '001',  '000',  '001'], 
             'w2': [ '101',  '111',  '010',  '111',  '001',  '111',  '100'],}

# generate test data frame and save as txt
test_raw_df = pd.DataFrame(data=test_data)
np.savetxt(r'test_vectors.txt', test_raw_df.values, fmt='%s')

# sign extend unsigned data with 0, since all data is expected to be sigend from this point on
test_sign_ext_df = test_raw_df.apply(lambda x: '0'+x if (x.name == 'a1' or x.name == 'a2') else x)
# str to int conversion
test_df = test_sign_ext_df.applymap(lambda x: BitArray(bin=x).int)
display(test_df)

,a1,a2,w1,w2
0,10,7,1,-3
1,15,15,-1,-1
2,1,0,2,2
3,9,9,0,-1
4,9,9,1,1
5,1,0,0,-1
6,13,1,1,-4


### Run simulation, then read in output results
* run the simulation
* get the results from txt file
* interpret strings in txt file as hexadecimal signed integers
* shove everything into dataframe

In [286]:
# trying to auto-run vivado simulation by calling some scripts. Does not work right now, you need to run simulation manually
# run simulation
#callProcess = subprocess.Popen('E:\Programme\Vivado\2019.2\bin\vivado.bat')
#print(callProcess)

In [45]:
# read in results from testbench
result_df_raw = pd.read_csv("output_results.txt", header=None, delim_whitespace=True, dtype=object)

# str to signed int conversion
result_df = pd.DataFrame()
result_df = result_df_raw.applymap(lambda x: BitArray(bin=x).int)
display(result_df)

,0,1,2,3
0,10,7,-30,-21
1,-15,-15,-15,-15
2,2,0,2,0
3,0,0,-9,-9
4,9,9,9,9
5,0,0,-1,0
6,13,1,-52,-4


### Generate Ground Truth and check if it matches with the output results

In [46]:
cols = ['a1w1','a2w1','a1w2','a2w2']
ground_truth_df = pd.DataFrame(columns=cols)

ground_truth_df["a1w1"] = test_df['a1'] * test_df['w1']
ground_truth_df["a2w1"] = test_df['a2'] * test_df['w1'] 
ground_truth_df["a1w2"] = test_df['a1'] * test_df['w2'] 
ground_truth_df["a2w2"] = test_df['a2'] * test_df['w2'] 
ground_truth_df

if(np.array_equal(ground_truth_df.values, result_df.values)):
    print("Result matches ground truth!")
else:
    print("\x1b[31mResult does not match ground truth!\x1b[0m")
    display(ground_truth_df)
    

Result matches ground truth!


## TCL Test Stuff
Not working right now


In [2]:
root = tkinter.Tcl()
tcl_script = """
# tcl script for redirecting 'puts' stream to python 
rename puts original_puts 
proc puts {args} {
    if {[llength $args] == 1} {
        return "TCL=> [lindex $args 0]"
    } else {
        eval original_puts $args
    }
}
"""


# call the tkinter tcl interpreter
root.tk.call('eval', tcl_script)
out = root.tk.eval('puts "test"')
print(out)
root.mainloop()

TCL=> test
